In [ ]:
# Clone repository
%cd /content
!rm -rf Deep_Learning_Project_Gil_Alon
!git clone https://github.com/gil-attar/Deep_Learning_Project_Gil_Alon.git
%cd Deep_Learning_Project_Gil_Alon

In [ ]:
# Install dependencies
!pip install ultralytics roboflow pyyaml -q

In [ ]:
# Verify GPU is available
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Download Dataset

In [ ]:
import os

# Set Roboflow API key
os.environ["ROBOFLOW_API_KEY"] = "zEF9icmDY2oTcPkaDcQY"

# Download dataset
!python scripts/download_dataset.py --output_dir data/raw

In [ ]:
# Verify dataset
!echo "Train: $(ls data/raw/train/images/ | wc -l) images"
!echo "Valid: $(ls data/raw/valid/images/ | wc -l) images"
!echo "Test: $(ls data/raw/test/images/ | wc -l) images"

## 3. Prepare Training Configuration

Create a data.yaml with correct paths for Colab.

In [ ]:
import yaml
from pathlib import Path

# Load original data.yaml to get class names
with open('data/raw/data.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Create training config with absolute paths
train_config = {
    'path': '/content/Deep_Learning_Project_Gil_Alon/data/raw',
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'names': config['names'],
    'nc': len(config['names'])
}

# Save training config
with open('data/train_config.yaml', 'w') as f:
    yaml.dump(train_config, f, default_flow_style=False)

print("Training config saved to data/train_config.yaml")
print(f"Number of classes: {train_config['nc']}")
print(f"Classes: {train_config['names'][:5]}..." if len(train_config['names']) > 5 else f"Classes: {train_config['names']}")

## 4. Train YOLOv8 (CNN Baseline)

Using YOLOv8n (nano) - smallest and fastest variant.

Training parameters:
- **Epochs:** 50 (can adjust based on convergence)
- **Image size:** 640x640
- **Batch size:** Auto (based on GPU memory)

In [ ]:
from ultralytics import YOLO

# Load pretrained YOLOv8n
yolo_model = YOLO('yolov8n.pt')

# Train on our dataset
yolo_results = yolo_model.train(
    data='data/train_config.yaml',
    epochs=50,
    imgsz=640,
    batch=-1,  # Auto batch size
    patience=10,  # Early stopping
    save=True,
    project='runs/train',
    name='yolov8n_baseline',
    exist_ok=True,
    pretrained=True,
    optimizer='auto',
    verbose=True,
    seed=42
)

In [ ]:
# Save best weights to models folder
import shutil
from pathlib import Path

Path('models').mkdir(exist_ok=True)

# Copy best weights
yolo_best = Path('runs/train/yolov8n_baseline/weights/best.pt')
if yolo_best.exists():
    shutil.copy(yolo_best, 'models/yolov8n_baseline.pt')
    print("✓ YOLOv8n weights saved to models/yolov8n_baseline.pt")
else:
    print("Warning: best.pt not found")

## 5. Train RT-DETR (Transformer Baseline)

Using RT-DETR-l (large) - best balance of speed and accuracy.

Same training parameters as YOLOv8 for fair comparison.

In [ ]:
from ultralytics import RTDETR

# Load pretrained RT-DETR-l
rtdetr_model = RTDETR('rtdetr-l.pt')

# Train on our dataset
rtdetr_results = rtdetr_model.train(
    data='data/train_config.yaml',
    epochs=50,
    imgsz=640,
    batch=-1,  # Auto batch size
    patience=10,  # Early stopping
    save=True,
    project='runs/train',
    name='rtdetr_baseline',
    exist_ok=True,
    pretrained=True,
    optimizer='auto',
    verbose=True,
    seed=42
)

In [ ]:
# Save best weights
rtdetr_best = Path('runs/train/rtdetr_baseline/weights/best.pt')
if rtdetr_best.exists():
    shutil.copy(rtdetr_best, 'models/rtdetr_baseline.pt')
    print("✓ RT-DETR weights saved to models/rtdetr_baseline.pt")
else:
    print("Warning: best.pt not found")

## 6. Quick Validation

Run inference on a few test images to verify models work.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Get 3 test images
test_images = list(Path('data/raw/test/images').glob('*.jpg'))[:3]

# Load trained models
yolo_trained = YOLO('models/yolov8n_baseline.pt')
rtdetr_trained = RTDETR('models/rtdetr_baseline.pt')

fig, axes = plt.subplots(3, 3, figsize=(15, 15))

for i, img_path in enumerate(test_images):
    # Original image
    img = Image.open(img_path)
    axes[i, 0].imshow(img)
    axes[i, 0].set_title('Original')
    axes[i, 0].axis('off')
    
    # YOLOv8 prediction
    yolo_result = yolo_trained.predict(img_path, verbose=False)[0]
    yolo_img = yolo_result.plot()
    axes[i, 1].imshow(yolo_img)
    axes[i, 1].set_title(f'YOLOv8 ({len(yolo_result.boxes)} detections)')
    axes[i, 1].axis('off')
    
    # RT-DETR prediction
    rtdetr_result = rtdetr_trained.predict(img_path, verbose=False)[0]
    rtdetr_img = rtdetr_result.plot()
    axes[i, 2].imshow(rtdetr_img)
    axes[i, 2].set_title(f'RT-DETR ({len(rtdetr_result.boxes)} detections)')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.savefig('evaluation/baseline_comparison.png', dpi=150)
plt.show()
print("\n✓ Saved comparison to evaluation/baseline_comparison.png")

## 7. Training Summary

In [ ]:
print("=" * 60)
print("STEP 3 COMPLETE: Baseline Model Training")
print("=" * 60)

print("\nModels trained:")
print(f"  ✓ YOLOv8n: models/yolov8n_baseline.pt")
print(f"  ✓ RT-DETR-l: models/rtdetr_baseline.pt")

print("\nTraining logs:")
print(f"  - runs/train/yolov8n_baseline/")
print(f"  - runs/train/rtdetr_baseline/")

print("\nNext steps:")
print("  1. Download model weights from models/ folder")
print("  2. Step 4: Run evaluation on test set")
print("  3. Step 5: Occlusion difficulty analysis")

## 8. Download Weights

Download the trained model weights to commit to GitHub.

In [ ]:
# List model files
!ls -lh models/

# You can download from the Colab file browser (📁 icon)
# or use this to create a zip:
!zip -r models.zip models/
print("\n✓ Download models.zip from the file browser")